In [1]:
## Script to compute mean diurnal cycle of SKT from IFS, and aggregate it to a  to coarser grid 
## E. Dutra April 2023

import xarray as xr
import numpy as np
import gribscan
import os 
from netCDF4 import Dataset,num2date 
import pandas as pd
import datetime as dt 
import time
import sys
import pandas as pd 

import matplotlib.pylab as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator

import agg_spatial_cython as agg_spatial

In [2]:
def gen_output(fout,ftype):
   # create output file 
  if os.path.isfile(fout):
    os.remove(fout)
  nc = Dataset(fout,'w',format='NETCDF4')

  if ftype=="HR":
      latR = lat_regH
      lonR = lon_regH
  elif ftype == "LR":
      latR = lat_regL
      lonR = lon_regL
    
  # create dimensions 
  nc.createDimension('lat',len(latR))
  nc.createDimension('lon',len(lonR))
  nc.createDimension('time',1)
  
   
  # create dimensions variables 
  cvar = nc.createVariable('lat','f4',['lat',])
  cvar.units = "degrees_north"
  cvar.long_name = "latitude"
  cvar.standard_name = "latitude"
  cvar.axis= "Y" 
  cvar[:] = latR[:,0]
  
  cvar = nc.createVariable('lon','f4',['lon',])
  cvar.units = "degrees_east"
  cvar.long_name = "longitude"
  cvar.standard_name = "longitude"
  cvar.axis= "X" 
  cvar[:] = lonR[0,:]
  
  cvar = nc.createVariable('time','i4',['time',])
  cvar.units = f"hours since {dstart.strftime('%Y-%m-%d')}T00:00:00"
  cvar.long_name = "time"
  cvar.standard_name = "time"
  cvar.axis= "T"
  cvar.calendar = "standard"
   
  
  cvar = nc.createVariable('LST','f4',('time','lat','lon'),
                           fill_value=ZFILL,zlib=True,complevel=6,
                           least_significant_digit=2)
  cvar.long_name='LST'
  cvar.units='Celsius'
    
  cvar = nc.createVariable('pvalid','f4',('time','lat','lon'),
                           fill_value=ZFILL,zlib=True,complevel=6,
                           least_significant_digit=1)
  cvar.long_name='percent of valid pixels in average'
  cvar.units='0-100'
    
  cvar = nc.createVariable('NSLOT','f4',('time',))
  cvar.long_name='total number of slots processed'
  cvar.units='-'

  if ftype == "LR":
      cvar = nc.createVariable('LST_std','f4',('time','lat','lon'),
                           fill_value=ZFILL,zlib=True,complevel=6,
                           least_significant_digit=2)
      cvar.long_name='LST_standard_deviation'
      cvar.units='Celsius'
    
      cvar = nc.createVariable('npgrid','f4',('lat','lon'),
                           fill_value=ZFILL,zlib=True,complevel=6,
                           least_significant_digit=2)
      cvar.long_name='number_points_grid'
      cvar.units='-'
    
  return nc

def inter2D(xIN):
  nn_interpolation = NearestNDInterpolator(points_ifs, xIN)
  return nn_interpolation(lon_regH, lat_regH)

In [3]:
# Generic definitions
tcc_min = 0.3
ZFILL=-999
DFOUT="/scratch/b/b381666/SKT_DIAG/"
exp="coarse"
dstart=dt.datetime(2020,6,1)
dend = dt.datetime(2020,8,31)

## Define output high res regular grid 
dlatH=-0.05;dlonH=0.05
lon_regH, lat_regH = np.meshgrid(np.arange(-80,80.05,0.05), np.arange(80,-80.05,-0.05))
## Define output low res regular grid
dlatL=-0.25;dlonL=0.25
lon_regL, lat_regL = np.meshgrid(np.arange(-70,70.25,0.25), np.arange(70,-70.25,-0.25));


#ncH.close()
#ncL.close()

In [4]:
t0_ = time.time()
if exp == 'coarse':
  datazarr = "/work/bm1235/b381679/c3/nemo_28km/TCo399_nemo_fdb_all1/json.dir/atm2d.json" 
elif exp ==   'nemo':
  datazarr='/work/bm1235/b381679/rundir/tco1279l137/hz9o/hres/intel.levante.hpcx/lvt.intel.sp/TCo1279_nemo_fdb_all/json.dir/atm2d.json' 
  ficmgg="/work/bm1235/ifs-inputs_nxg_c3/tco1279l137/hz9o/LWDA/2020012000/gfc/ICMGGhz9oINIT"
elif exp == 'feson':
  datazarr ='/scratch/a/a270046/rundir/tco1279l137/hz9o/hres/intel.levante.openmpi/lvt.intel.sp/TCo1279_fesom_fdb_all//json.dir/atm2d.json' 
  # grib_copy -w shortName=lsm/cl,dataDate=20200120 /work/bm1235/ifs-inputs_nxg_c3/tco1279l137/hz9o/LWDA/2020012000/gfc/ICMGGhz9oINIT /scratch/b/b381666/ini_tco1279
  ficmgg="/scratch/b/b381666/ini_tco1279"
print("Loading: ",datazarr)
data = xr.open_zarr("reference::"+datazarr, consolidated=False)
print(f"loaded data list {datazarr} in {time.time()-t0_:.1f} sec") 
#display(data)

Loading:  /work/bm1235/b381679/c3/nemo_28km/TCo399_nemo_fdb_all1/json.dir/atm2d.json
loaded data list /work/bm1235/b381679/c3/nemo_28km/TCo399_nemo_fdb_all1/json.dir/atm2d.json in 11.4 sec


In [5]:
dsINI = xr.open_dataset(ficmgg, engine='cfgrib',backend_kwargs={'indexpath':''})
land_inlandw = (dsINI.lsm.values >= 0.5) | ( dsINI.cl.values >= 0.5 )


NameError: name 'ficmgg' is not defined

In [6]:
## Get the grid 
model_lon = np.where(data.lon.values>180, data.lon.values-360, data.lon.values)
model_lat = data.lat.values

## Select region of interest 
reg = ( (model_lat>-81) & (model_lat<81) &
         (model_lon >-81) & (model_lon<81) )
npp = np.sum(reg)
points_ifs = np.vstack((model_lon[reg], model_lat[reg])).T

In [7]:
dlist = pd.date_range(start=dstart,end=dend,freq='d')
#dlist = pd.date_range(start=dstart,end=dt.datetime(2020,6,10),freq='d')
print(f"will process {npp} points for {len(dlist)} days")

will process 290882 points for 92 days


In [8]:
## pre-allocate aggregation
# nearest LR grid 
NiY =  (( model_lat[reg] - (lat_regL[0,0]-0.5*dlatL )) // dlatL ).astype('i4') 
NiX =  (( model_lon[reg] - (lon_regL[0,0]-0.5*dlonL )) // dlonL ).astype('i4') 

# main loop on hours 
for ih in range(24):
    t0H = time.time()
    
    ## Generate output files
    foutH = f"{DFOUT}/NETCDF4_{exp}_LST_{'test'}_HR_{ih:02d}.nc"
    foutL = f"{DFOUT}/NETCDF4_{exp}_LST_{'test'}_LR_{ih:02d}.nc"
    ncH = gen_output(foutH,'HR')
    ncL = gen_output(foutL,'LR')
    
    xAGG    = np.zeros(lat_regL.shape,dtype='f4')
    xCOUNT  = np.zeros(lat_regL.shape,dtype='i4')
    xMISS   = np.zeros(lat_regL.shape,dtype='f4')
    xAGGSTD = np.zeros(lat_regL.shape,dtype='f4')
    
    nslotSTP = 0
    zAVG = np.zeros(npp,'f4')
    zVAL = np.zeros(npp,'f4')
    # loop on days of month 
    for idate in dlist:
        slot = idate+dt.timedelta(hours=ih)
        nslotSTP = nslotSTP + 1
        print("loading",slot)
        #load data into memory 
        skt = data.skt.sel(time=slot)[reg] - 273.16
        tcc = data.tcc.sel(time=slot)[reg]
    
        # select only clear sky 
        xOK = tcc <= tcc_min
        zVAL[xOK] = zVAL[xOK] + 1 
        zAVG[xOK] = zAVG[xOK] + skt[xOK]   
    #compute averages 
    zAVG = np.where(zVAL>0,zAVG / zVAL,ZFILL)
    zVAL = np.where(zVAL>0,100*zVAL/nslotSTP,0)   
    
    # write to output HR 
    ncH.variables['time'][0] = ih
    ncH.variables['NSLOT'][0] = nslotSTP
    ncH.variables['LST'][0,:,:] = inter2D(zAVG)
    ncH.variables['pvalid'][0,:,:] = inter2D(zVAL)
  
    # spatial aggregations
    ## LST 
    # first mask ocean points 
    zAVG[~land_inlandw[reg]] = ZFILL
    zVAL[~land_inlandw[reg]] = 0
    
    agg_spatial.agg_pixels(NiX,NiY,zAVG.astype('f4'),xAGG,xAGGSTD,xCOUNT,ZFILL)
    ncL.variables['time'][0] = ih
    ncL.variables['NSLOT'][0] = nslotSTP
    ncL.variables['LST'][0,:,:] = xAGG
    ncL.variables['LST_std'][0,:,:] = xAGGSTD
    if ih == 0: ncL.variables['npgrid'][:,:] = xCOUNT 
    
    ## pvalid 
    agg_spatial.agg_pixels(NiX,NiY,zVAL.astype('f4'),xAGG,xAGGSTD,xCOUNT,ZFILL)
    ncL.variables['pvalid'][0,:,:] = xAGG
    
    
 
    ncH.close()
    ncL.close()  
    
    print(f"Processed {dstart} hour {ih} with {nslotSTP} slots in {time.time()-t0H:.1f} sec")

loading 2020-06-01 00:00:00
loading 2020-06-02 00:00:00
loading 2020-06-03 00:00:00
loading 2020-06-04 00:00:00
loading 2020-06-05 00:00:00
loading 2020-06-06 00:00:00
loading 2020-06-07 00:00:00
loading 2020-06-08 00:00:00
loading 2020-06-09 00:00:00
loading 2020-06-10 00:00:00
loading 2020-06-11 00:00:00
loading 2020-06-12 00:00:00
loading 2020-06-13 00:00:00


KeyboardInterrupt: 

In [11]:
skt.values
#zAVG[~land_inlandw[reg]] = ZFILL
#agg_spatial.agg_pixels(NiX,NiY,zAVG.astype('f4'),xAGG,xAGGSTD,xCOUNT,ZFILL)

#plt.imshow(xCOUNT)
#data.skt.sel(time=slot).values
#data
#plt.scatter(model_lon[reg][0:-1:2],model_lat[reg][0:-1:2],c=land_inlandw[reg][0:-1:2])
#data.lsm
#type(skt.values)

array([  0.10713562,   0.11104187,   0.09541687, ..., -42.12723938,
       -42.72489563, -43.31473938])